In [ ]:
"""
Created on Fri May 02 16:38 2025

Prepare the fronts of the different basins to apply the melt param in ice-shelf areas not resolved at 1 deg

@author: Clara Burgard
"""

In [60]:
import xarray as xr
import numpy as np
from cdo import Cdo
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from datetime import date
import xesmf as xe
import cc3d

In [4]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [5]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 2.1.0


In [114]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
inputpath_interim2 = '/data/cburgard/MELT_PARAM_NONRESOLVED/interim/'

In [7]:
file_Justine = xr.open_dataset(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc').drop('quantile')

In [125]:
BedMachine_mask = xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_masks_only.nc')
BedMachine_vars_2km= xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_allvars_withcategories.nc')

In [8]:
fNEMO_mask = 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'

In [9]:
input_grid = xr.open_dataset(inputpath_interim + 'IMBIE_2km_stereo_withbnds.nc')
# will be provided, was created with create_cdo_stereo_grid.ipynb

TRY DEFINING THE BORDER OF THE UNRESOLVED ICE SHELVES 

In [19]:
domain_cfg = xr.open_dataset(inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')

In [11]:
masks_for_eORCA1 = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1.nc')

In [14]:
masks_for_eORCA1['mask_isf_open'].plot()

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


HAVE JUSTINE'S MASK ON NEMO GRID

In [ ]:
cdo selvar,isf_draft domain_cfg_withclosedlargeisf.nc isfdraft_withclosedlargeisf.nc
cdo selvar,isf_draft -setgrid,domain_cfg_withclosedlargeisf.nc domain_cfg_withopenisf.nc isfdraft_withopenisf.nc

cdo remapnn,Iceshelf_extrap_IMBIE2_v2.nc isfdraft_withclosedlargeisf.nc isfdraft_withclosedlargeisf_onIMBIEgrid.nc 
cdo remapnn,Iceshelf_extrap_IMBIE2_v2.nc isfdraft_withopenisf.nc isfdraft_withopenisf_onIMBIEgrid.nc 

In [29]:
isfmask_nn = cdo.remapnn(inputpath_interim + 'NEMO_grid_withbnds.nc',input=file_Justine['Iceshelf_extrap'], returnArray='Iceshelf_extrap')
isfmask = xr.DataArray(data=isfmask_nn, dims=['y','x'])

regrid NEMO stuff on Justine's grid and then also apply a lsmask on Justine's stuff and this gives us the rest

In [17]:
ID_open_list = [21,66,67,117,124,125,127,128]

In [28]:
file_Justine['Iceshelf_extrap'].plot()

In [21]:
tweaked_domaing_cfg = domain_cfg.copy()
for id_open in tqdm(ID_open_list):
    for vv in ['bottom_level','top_level','isf_draft','bathy_metry']:
        tweaked_domaing_cfg[vv] = tweaked_domaing_cfg[vv].where(masks_for_eORCA1['mask_isf'] != id_open, 0)

  0%|          | 0/8 [00:00<?, ?it/s]

In [31]:
tweaked_domaing_cfg.to_netcdf(inputpath_interim + 'domain_cfg_withclosedlargeisf.nc')

In [39]:
domain_cfg.to_netcdf(inputpath_interim + 'domain_cfg_withopenisf.nc')

In [40]:
largeisf_on_IMBIEgrid = xr.open_dataset(inputpath_interim + 'isfdraft_withopenisf_onIMBIEgrid.nc')

In [ ]:
unresolved_areas = ((largeisf_on_IMBIEgrid['isf_draft'] == 0) & (BedMachine_mask['isf_conc'] > 0))

In [58]:
onlyunresolvedareas = file_Justine['Iceshelf_extrap'] * np.nan
for id_open in tqdm(ID_open_list):
    unresolved_kisf = file_Justine['Iceshelf_extrap'].where((file_Justine['Iceshelf_extrap'] == id_open) & (largeisf_on_IMBIEgrid['isf_draft'] == 0) & (BedMachine_mask['isf_conc'] > 0))
    onlyunresolvedareas = onlyunresolvedareas.combine_first(unresolved_kisf)

  0%|          | 0/8 [00:00<?, ?it/s]

In [56]:
# Correct manually to remove everything that's not touching ground
xx = onlyunresolvedareas.x
yy = onlyunresolvedareas.y
#onlyunresolvedareas = onlyunresolvedareas.where(~((xx >= -2.23e6) & (xx <= -2.00e6) & (yy >= 1.12e6) & (yy <= 1.24e6)), np.nan)
#onlyunresolvedareas = onlyunresolvedareas.where(~((xx >= -7.6e5) & (xx <= -6.88e5) & (yy >= 1.35e6) & (yy <= 1.7e6)), np.nan)
#onlyunresolvedareas = onlyunresolvedareas.where(~((xx >= -6.77e5) & (xx <= -5.4e5) & (yy >= 1.57e6) & (yy <= 1.88e6)), np.nan)
#onlyunresolvedareas = onlyunresolvedareas.where(~((xx >= -5.5e5) & (xx <= -4.9e5) & (yy >= 1.69e6) & (yy <= 1.91e6)), np.nan)

In [64]:
mask_unresolved = (onlyunresolvedareas > 1).squeeze('time')

In [67]:
threshold = 1
connectivity = 4

#find connected components
dusted = cc3d.dust(mask_unresolved.values.astype(np.int64), 
           threshold = threshold, 
           connectivity = connectivity, 
           in_place = False)

In [76]:
labels_out = cc3d.connected_components(dusted, 
                               connectivity = connectivity)

In [141]:
unresolved_components = xr.DataArray(data=labels_out, dims=mask_unresolved.dims)

In [142]:
unresolved_components = unresolved_components.where(unresolved_components > 0)+133
#unresolved_components = unresolved_components.where(np.isfinite(unresolved_components), 0)

In [143]:
allbasins = unresolved_components.combine_first(file_Justine['Iceshelf_extrap']).to_dataset(name='Iceshelf_extrap')

In [144]:
ice_draft = (BedMachine_vars_2km['thickness'] - BedMachine_vars_2km['surface']).sel(category=3).drop('category')

In [145]:
zmin_list = []
zmax_list = []
for kisf in tqdm(range(1,818)):
    isf_mask = (allbasins['Iceshelf_extrap'] == kisf) & (BedMachine_mask['isf_conc'] > 0)

    zmin = ice_draft.where(isf_mask).min()
    zmin_list.append(zmin.assign_coords({'ID': kisf}))
    zmax = ice_draft.where(isf_mask).max()
    zmax_list.append(zmax.assign_coords({'ID': kisf}))

zmin_all = xr.concat(zmin_list, dim='ID')
zmax_all = xr.concat(zmax_list, dim='ID')

  0%|          | 0/817 [00:00<?, ?it/s]

In [146]:
allbasins['zmin'] = zmin_all
allbasins['zmax'] = zmax_all

In [147]:
allbasins.to_netcdf(inputpath_interim2 + 'allbasins_IMBIEgrid.nc')

In [ ]:
cdo remapnn,NEMO_grid_withbnds.nc -selvar,Iceshelf_extrap allbasins_IMBIEgrid.nc allbasins_NEMO1grid.nc

In [148]:
check = xr.open_dataset(inputpath_interim2 + 'allbasins_NEMO1grid.nc')

In [150]:
# need to define the fronts and assign the zmin and zmax. Before that, group the scattered things in regions that make more sense. 
# But I think this will take quite some time, manually :/

In [149]:
check['Iceshelf_extrap'].plot()

In [103]:
#Put some order in all scattered components => will do that at another time
plt.figure()
unresolved_components.where((unresolved_components >= 1) & (unresolved_components <= 10), drop=True).plot(vmin=1,vmax=100)

[1, 19]

In [71]:


labelled_isf = xr.DataArray(labels_out, 
                coords = {"y": file_conc.y, "x": file_conc.x}, 
                dims = ["y", "x"],
                name = "labels")

all_isf_list = np.array(list(new_mask.groupby(new_mask).groups))
isf_labels = all_isf_list[all_isf_list>9]

NameError: name 'file_conc' is not defined

give numbers to all these subbasins, then regrid on eORCA1 and put them into the mask file

In [59]:
file_Justine.ID

<xarray.DataArray 'ID' (ID: 133)>
array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133])
Coordinates:
  * ID       (ID) int64 1 2 3 4 5 6 7 8 9 ... 126 127 128 129 130 131 132 133

In [ ]:
for id_open in tqdm(ID_open_list):
    largeisf_on_IMBIEgrid['isf_draft'].where()

In [11]:
ds_isfNEMO = xr.Dataset()

ds_isfNEMO['mask_isf'] = isfmask.where(domain_cfg['isf_draft'] > 0)
ds_isfNEMO['mask_isf'].attrs['standard_name'] = 'mask of all the ice shelves'
ds_isfNEMO['mask_isf'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

#ds_isfNEMO['mask_isf_onlyisfconc1'] = ds_isfNEMO['mask_isf'].where(isfconc['isf_conc'] > 0.9999)


In [12]:
ds_isfNEMO['floating_frac'] = remapped_isf_conc # .where(isfconc['isf_conc'] < 0.9999,1)

NameError: name 'remapped_isf_conc' is not defined

CLOSE ALL ISF TO IDENTIFY THEIR FRONTS

In [ ]:
## gives the list of names when needed to better understand stuff
#for iid in file_Justine.ID:
#    print(iid.values,file_Justine['NAME'].sel(ID=iid).values)

In [12]:
file_Justine_closed = file_Justine

In [13]:
# Prepare domain_cfg closing (setting to 0) all information about the closed cavities
domain_cfg_closed =  domain_cfg.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    for vv in ['bottom_level','top_level','isf_draft','bathy_metry']:
        domain_cfg_closed[vv] = domain_cfg_closed[vv].where(ds_isfNEMO['mask_isf'] != id_closed.values, 0)

  0%|          | 0/133 [00:00<?, ?it/s]

In [14]:
# Prepare mask of only closed cavities 
mask_isf_closed = ds_isfNEMO['mask_isf']

In [15]:
ds_isfNEMO['mask_isf_closed'] = mask_isf_closed
ds_isfNEMO['mask_isf_closed'].attrs['standard_name'] = 'mask of the ice shelves that are closed'
ds_isfNEMO['mask_isf_closed'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

IDENTIFY FRONT

In [16]:
# Create a mask discriminating between land, ocean and ice shelf
mask_0_1_2 = ds_isfNEMO['mask_isf'] * 2
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] != 0, 400) # land
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] == 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(domain_cfg['isf_draft'] == 0, 200) # ice shelf

In [17]:
# Create a mask of the extrapolated domains and only keep the domains corresponding to the closed cavities
extrap_domains_closed = isfmask.copy()

#for kisf in ID_open_list:
#    mask_0_1_2_closed = mask_0_1_2.where(ds_isfNEMO['mask_isf'] != kisf, 400)
#    extrap_domains_closed = extrap_domains_closed.where(extrap_domains_closed != kisf, np.nan)

In [18]:
# set all ice shelves to 300
mask_front0 = mask_0_1_2.where((mask_0_1_2 == 0) | (mask_0_1_2 == 400), 300).copy()

In [20]:
mask_front = mask_front0.copy()
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
mask_front = mask_front.where((mask_front0.shift(x=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(x=1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=1)-mask_front0)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,extrap_domains_closed)

For the ice shelves not resolved at all in eORCA1

In [21]:
# define the domains that have not an associated front yet
extrap_domains_double_closed = extrap_domains_closed.copy()

for kisf in file_Justine_closed.ID:
    if (mask_front == kisf).astype(int).sum() > 0:
        #print(kisf.values)
        extrap_domains_double_closed = extrap_domains_double_closed.where(extrap_domains_double_closed != kisf, np.nan).drop('ID')

In [22]:
# To double check where the front points are
#iid = 89
#plt.figure()
#((mask_0_2) + (isfmask['Iceshelf_extrap'] == iid)).plot()
#plt.title(str(iid))

In [23]:
# set all ice shelves to land to have a delimitation of the whole contour of Antarctica
mask_0_2 = mask_0_1_2.where(mask_0_1_2 != 200, 2)
mask_0_2 = mask_0_2.where(mask_0_2 != 400, 2)

mask_front_new = mask_0_2.copy()

# check all directions and set points at border between ocean and land(300-0) to 500
mask_front_new = mask_front_new.where((mask_0_2.shift(x=-1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(x=1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(y=-1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(y=1)-mask_0_2) <= 0,5)
# cut out all front points south of the y = 90 latitude and in the ocean domain
mask_front_new = mask_front_new.where((mask_front_new == 5) & (mask_front_new.y < 90) & (mask_0_2 == 0))
# set the ice shelf number of the extrapolated domain
mask_front_new = mask_front_new.where(mask_front_new != 5,extrap_domains_double_closed)

In [24]:
# check that it is the right delimitation (i.e. on the ocean points)
((mask_front_new > 0) + mask_0_2).plot()

In [29]:
mask_0_2.plot()

In [25]:
# combining the masks of the fronts (mask_front and mask_front_new)
mask_front_all = mask_front.copy()
for iid in file_Justine_closed.ID:
    if (ds_isfNEMO['mask_isf_closed'] == iid).astype(int).sum().values == 0: # if the ice shelf is not resolved in your grid, add the mask front inferred for them
        #print(iid.values, file_Justine_closed['NAME'].sel(ID=iid).values, (mask_front_new == iid).astype(int).sum().values)
        mask_front_all = mask_front_all.combine_first(mask_front_new.where(mask_front_new == iid))

In [31]:
mask_front_file = xr.Dataset()
mask_front_file['mask_front'] = mask_front_all
mask_front_file['ID'] = file_Justine['ID']
mask_front_file.to_netcdf(inputpath_interim + 'mask_fronts_ALLISF_eORCA1.nc')

In [32]:
mask_front_file['mask_front'].plot()